# 模型参数的延后初始化
如果做了上⼀节练习，你会发现模型net在调⽤初始化函数initialize之后、在做前向计
算net(X)之前时，权重参数的形状中出现了0。虽然直觉上initialize完成了所有参数初始
化过程，然而这在Gluon中却是不⼀定的。我们在本节中详细讨论这个话题。
## 延后初始化
也许读者早就注意到了，在之前使⽤Gluon创建的全连接层都没有指定输⼊个数。例如，在上⼀节
使⽤的多层感知机net⾥，我们创建的隐藏层仅仅指定了输出⼤小为256。当调⽤initialize函
数时，由于隐藏层输⼊个数依然未知，系统也⽆法得知该层权重参数的形状。只有在当我们将
形状是(2, 20)的输⼊X传进⽹络做前向计算net(X)时，系统才推断出该层的权重参数形状为(256,
20)。因此，这时候我们才能真正开始初始化参数。

让我们使⽤上⼀节中定义的MyInit类来演⽰这⼀过程。我们创建多层感知机，并使⽤MyInit实
例来初始化模型参数。

In [1]:
from mxnet import nd, init
from mxnet.gluon import nn
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        # 实际的初始化逻辑在此省略了
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'),
        nn.Dense(10))
net.initialize(init = MyInit())

注意，虽然MyInit被调⽤时会打印模型参数的相关信息，但上⾯的initialize函数执⾏完并
未打印任何信息。由此可⻅，调⽤initialize函数时并没有真正初始化参数。下⾯我们定义输
⼊并执⾏⼀次前向计算

In [3]:
X = nd.random.uniform(shape=(2,20))
y=net(X)

这时候，有关模型参数的信息被打印出来。在根据输⼊X做前向计算时，系统能够根据输⼊的形
状⾃动推断出所有层的权重参数的形状。系统在创建这些参数之后，调⽤MyInit实例对它们进
⾏初始化，然后才进⾏前向计算。
当然，这个初始化只会在第⼀次前向计算时被调⽤。之后我们再运⾏前向计算net(X)时则不会
重新初始化，因此不会再次产⽣MyInit实例的输出。

In [4]:
y=net(X)

系统将真正的参数初始化延后到获得⾜够信息时才执⾏的⾏为叫作延后初始化（deferred initialization）。
它可以让模型的创建更加简单：只需要定义每个层的输出⼤小，而不⽤⼈⼯推测它们
的输⼊个数。这对于之后将介绍的定义多达数⼗甚⾄数百层的⽹络来说尤其⽅便。
然而，任何事物都有两⾯性。正如本节开头提到的那样，延后初始化也可能会带来⼀定的困惑。
在第⼀次前向计算之前，我们⽆法直接操作模型参数，例如⽆法使⽤data函数和set_data函数
来获取和修改参数。因此，我们经常会额外做⼀次前向计算来迫使参数被真正地初始化。
## 避免延后初始化
如果系统在调⽤initialize函数时能够知道所有参数的形状，那么延后初始化就不会发⽣。我
们在这⾥分别介绍两种这样的情况。
第⼀种情况是我们要对已初始化的模型重新初始化时。因为参数形状不会发⽣变化，所以系统能
够⽴即进⾏重新初始化。

In [5]:
net.initialize(init=MyInit(), force_reinit=True)

Init dense0_weight (256, 20)
Init dense1_weight (10, 256)


第⼆种情况是我们在创建层的时候指定了它的输⼊个数，使系统不需要额外的信息来推测参数形
状。下例中我们通过in_units来指定每个全连接层的输⼊个数，使初始化能够在initialize函
数被调⽤时⽴即发⽣。

In [6]:
net = nn.Sequential()
net.add(nn.Dense(256, in_units=20, activation='relu'))
net.add(nn.Dense(10, in_units=256))
net.initialize(init=MyInit())

Init dense2_weight (256, 20)
Init dense3_weight (10, 256)


# 小结
- 系统将真正的参数初始化延后到获得⾜够信息时才执⾏的⾏为叫作延后初始化。
- 延后初始化的主要好处是让模型构造更加简单。例如，我们⽆须⼈⼯推测每个层的输⼊个数。
- 也可以避免延后初始化。